<h1>Table of Contents<span class="tocSkip"></span></h1>
<div class="toc"><ul class="toc-item"><li><span><a href="#Import-data" data-toc-modified-id="Import-data-1"><span class="toc-item-num">1&nbsp;&nbsp;</span>Import data</a></span><ul class="toc-item"><li><span><a href="#Import-UK-Biobank-ages" data-toc-modified-id="Import-UK-Biobank-ages-1.1"><span class="toc-item-num">1.1&nbsp;&nbsp;</span>Import UK Biobank ages</a></span></li></ul></li><li><span><a href="#Functions" data-toc-modified-id="Functions-2"><span class="toc-item-num">2&nbsp;&nbsp;</span>Functions</a></span><ul class="toc-item"><li><span><a href="#Overall-simulation-function" data-toc-modified-id="Overall-simulation-function-2.1"><span class="toc-item-num">2.1&nbsp;&nbsp;</span>Overall simulation function</a></span></li></ul></li><li><span><a href="#Generate-the-simulated-data" data-toc-modified-id="Generate-the-simulated-data-3"><span class="toc-item-num">3&nbsp;&nbsp;</span>Generate the simulated data</a></span><ul class="toc-item"><li><span><a href="#Simulated-mCAs-(each-simulation-with-a-different-fitness-effect)" data-toc-modified-id="Simulated-mCAs-(each-simulation-with-a-different-fitness-effect)-3.1"><span class="toc-item-num">3.1&nbsp;&nbsp;</span>Simulated mCAs (each simulation with a different fitness effect)</a></span></li><li><span><a href="#Simulated-high-mutation-rate-mCAs-(each-simulation-with-a-different-fitness-and-mu)" data-toc-modified-id="Simulated-high-mutation-rate-mCAs-(each-simulation-with-a-different-fitness-and-mu)-3.2"><span class="toc-item-num">3.2&nbsp;&nbsp;</span>Simulated high mutation rate mCAs (each simulation with a different fitness and mu)</a></span></li><li><span><a href="#Simulated-mLOX-with-2-different-fitness-effects-and-2-different-mutation-rates" data-toc-modified-id="Simulated-mLOX-with-2-different-fitness-effects-and-2-different-mutation-rates-3.3"><span class="toc-item-num">3.3&nbsp;&nbsp;</span>Simulated mLOX with 2 different fitness effects and 2 different mutation rates</a></span></li></ul></li></ul></div>

In [1]:
from IPython.core.display import display, HTML
display(HTML("<style>.container { width:80% !important; }</style>"))

In [2]:
# imported packages
import numpy as np
import pandas as pd
import csv
import copy

In [3]:
grey1 = '#f7f7f7'
grey2 = '#cccccc'
grey3 = '#969696'
grey4 = '#636363'
grey5 = '#252525'

In [4]:
seed = 3
np.random.seed(seed)

# Import data

## Import UK Biobank ages

In [5]:
#import UK biobank age data
biobank_ages = pd.read_csv('Data_files/UKBiobank_ages_information.tsv', sep='\t', low_memory=False)
biobank_ages

,pseudo_ID,Age
0,0,65.0
1,1,50.0
2,2,51.0
3,3,52.0
4,4,65.0
...,...,...
502408,502408,67.0
502409,502409,65.0
502410,502410,65.0
502411,502411,58.0


In [6]:
# age bins
biobank_ages_bins = biobank_ages['Age'].value_counts(). \
    reset_index().sort_values('index').rename(columns={'Age': 'Count',
                                                       'index': 'Age'}).reset_index(drop=True)
biobank_ages_bins

,Age,Count
0,37.0,1
1,38.0,1
2,39.0,5
3,40.0,5476
4,41.0,11055
5,42.0,11155
6,43.0,11908
7,44.0,12180
8,45.0,12655
9,46.0,12708


# Functions

In [7]:
# define how tau (symmetric division time) changes with time (if both tau1 and tau2 are the same, then it does not change with time)
# define everything in terms of YEARS (so need to pass t/lambda to this function)
def tau_of_t(t_years, tau1, tau2, t1, t2):
    if t_years < t1:
        return tau1
    elif t_years > t2:
        return tau2
    else:
        return tau1 + (t_years - t1) * (tau2 - tau1) / (t2 - t1) # = tau1 if tau1 and tau2 are both the same

In [8]:
# converts tau-dependence into s-dependence for integration
# t in years
def s_of_t(t, s0, tau_params):
    tau = tau_of_t(t, *tau_params)
    return s0 / tau

In [9]:
# calculate s_param given tau_params
def s_params_calc(s0, tau_params):
    tau1, tau2, t1, t2 = tau_params
    return s0/tau1, s0/tau2, t1, t2

In [10]:
# heuristic (approximate) variant density for variable tau (not relevant if not varying tau)
# do by converting variable tau into variable s (proportionally) and integrating (numerically)
# x is frequency
# lam*D0 is tau - this should equal 1 (removed in new version - used to be in expression for phi: np.exp(int_s*lam*D0))
# log_out gives natural log of the output (with higher accuracy than simply np.log(out))
def heuristic_mCA_rho(x, s_mCA, t, tau_params, N, c, log_out=False):
    int_s, err = it.quad(s_of_t, 0, t, args=(s_mCA, tau_params))

    phi = c * ((np.exp(int_s) - 1) / (2 * N * s_mCA))

    if log_out:
        rho = np.log(N / (1 - 2 * x)) - x / (phi * (1 - 2 * x))
        return rho

    else:
        rho = N / (1 - 2 * x) * np.exp(-x / (phi * (1 - 2 * x)))
        return rho

In [11]:
# define function 'divide'
def divide(clone_info, dt, B0, D0):
    # dt = a small interval of time,
    # D0 = symmetric rate to differentiated
    # B0 = symmetric rate to self renewal
    # B  = modified birth rate including fitness advantage

    mutations = clone_info[0]  # [(mutation_ID, fitness)]
    clone_size = clone_info[1]  # n
    fitness_effects = [i[1] for i in mutations]
    F = sum(fitness_effects)
    B = B0 * (1.0 + F)  # B = birth rate, which is 1 + the overall fitness effect of the mutations within a particular clone
    number_of_births = np.random.poisson(clone_size * B * dt)  # pulls a random number from the Poisson distribution with/
    # mean of clone_size x birth rate x interval of time
    number_of_deaths = np.random.poisson(clone_size * D0 * dt)
    new_clone_size = clone_size + number_of_births - number_of_deaths
    return [mutations, new_clone_size]

In [12]:
# define function 'mutation_fitness' of the form:
# sb = beneficial fitness effect
# sd = deleterious fitness effect
# Pb = probability of beneficial fitness effect
# Pn = probability of neutral fitness effect
# Pd = probability of deleterious fitness effect

def mutation_fitness(DFE):
    total_prob = sum([i[1] for i in DFE])
    normalized_DFE = [[i[0], i[1] / total_prob] for i in DFE] # DFE = [[sd, Pd], [0.0, Pn], [sb, Pb]]

    random_number = np.random.random()
    cumulative = 0.0
    for [s, P] in normalized_DFE:
        cumulative = cumulative + P
        if random_number < cumulative:
            fitness_effect = s
            break
    return fitness_effect

In [13]:
# define function 'pop_size'
def pop_size(pop):
    total=0
    for clone in pop:
        sub_pop_size=clone[-1]
        total=total+sub_pop_size
    return total

In [14]:
# define function 'sequence' (to include mCA variants)
def sequence(pop, mean_depth1, depth_SD1, mean_depth2, depth_SD2, s1_mCA):
    total_pop_size = pop_size(pop)  # used to be called N (which is a constant =1e5 in main code). Now called total_pop_size
    variant_dict = {}
    list_of_variants = []

    # first determine total freq of variant across all clones and record clones in which it arises
    for (clone1, size) in pop:
        if len(clone1) > 1:  # ensures non-WT clones only
            clone = copy.deepcopy(clone1)
            clone.pop(0)

            multi_mutant = len(clone) # number of mutations in clone (not including WT)
            for mutation in clone:

                # update mutations frequency
                mID = mutation[0]
                if mID in variant_dict:
                    variant_dict[mID]['true_freq'] += (size / total_pop_size) #cell fraction

                else:
                    variant_dict[mID] = {
                        'true_freq': (size / total_pop_size)}  # true cell fraction

                    if mutation[1] == s1_mCA:
                        mutation_type = 'mCA'
                    elif mutation[1]>0:
                        mutation_type = 'driver'
                    variant_dict[mID]['mutation_type'] = mutation_type

                    variant_dict[mID]['fitness'] = mutation[1]
                    variant_dict[mID]['multi_mutant'] = multi_mutant

    for m, e in variant_dict.items():

        mutation_type = e['mutation_type']
        true_freq = e['true_freq']
        fitness = e['fitness']
        multi_mutant = e['multi_mutant']

        # new method for making normal distribution of depths
        depth1 = int(np.random.normal(mean_depth1, depth_SD1))
        if depth1 <= 1:
            depth1 = 2
        reads1 = np.random.binomial(depth1, true_freq)

        depth2 = int(np.random.normal(mean_depth2, depth_SD2))
        if depth2 <= 1:
            depth2 = 2
        reads2 = np.random.binomial(depth2, true_freq)

        freq1 = reads1 / depth1
        freq2 = reads2 / depth2

        list_of_variants.append({"mutation_ID": m,
                                 "mutation_type": mutation_type,
                                 "fitness": fitness,
                                 "true_freq": true_freq,
                                 "multi_mutant":multi_mutant,
                                 "freq1": freq1,
                                 "reads1": reads1,
                                 "depth1": depth1,
                                 "freq2": freq2,
                                 "reads2": reads2,
                                 "depth2": depth2
                                 })
    return list_of_variants

In [15]:
# define function 'sequence' (to include mCA variants) - for 2 fitness effects an 2 mutation rates
def sequence_s1s2(pop, mean_depth1, depth_SD1, mean_depth2, depth_SD2, s1_mCA, s2_mCA):
    total_pop_size = pop_size(pop)  # used to be called N (which is a constant =1e5 in main code). Now called total_pop_size
    variant_dict = {}
    list_of_variants = []

    # first determine total freq of variant across all clones and record clones in which it arises
    for (clone1, size) in pop:
        if len(clone1) > 1:  # ensures non-WT clones only
            clone = copy.deepcopy(clone1)
            clone.pop(0)

            multi_mutant = len(clone) # number of mutations in clone (not including WT)
            for mutation in clone:

                # update mutations frequency
                mID = mutation[0]
                if mID in variant_dict:
                    variant_dict[mID]['true_freq'] += (size / total_pop_size) #cell fraction

                else:
                    variant_dict[mID] = {
                        'true_freq': (size / total_pop_size)}  # , true cell fraction

                    if mutation[1] == s1_mCA:
                        mutation_type = 'mCA_1'
                    if mutation[1] == s2_mCA:
                        mutation_type = 'mCA_2'
                    if mutation[1] not in [s1_mCA, s2_mCA]:
                        if mutation[1]>0:
                            mutation_type = 'driver'
                    variant_dict[mID]['mutation_type'] = mutation_type

                    variant_dict[mID]['fitness'] = mutation[1]
                    variant_dict[mID]['multi_mutant'] = multi_mutant

    for m, e in variant_dict.items():

        mutation_type = e['mutation_type']
        true_freq = e['true_freq']
        fitness = e['fitness']
        multi_mutant = e['multi_mutant']

        # new method for making normal distribution of depths
        depth1 = int(np.random.normal(mean_depth1, depth_SD1))
        if depth1 <= 1:
            depth1 = 2
        reads1 = np.random.binomial(depth1, true_freq)

        depth2 = int(np.random.normal(mean_depth2, depth_SD2))
        if depth2 <= 1:
            depth2 = 2
        reads2 = np.random.binomial(depth2, true_freq)

        freq1 = reads1 / depth1
        freq2 = reads2 / depth2

        list_of_variants.append({"mutation_ID": m,
                                 "mutation_type": mutation_type,
                                 "fitness": fitness,
                                 "true_freq": true_freq,
                                 "multi_mutant":multi_mutant,
                                 "freq1": freq1,
                                 "reads1": reads1,
                                 "depth1": depth1,
                                 "freq2": freq2,
                                 "reads2": reads2,
                                 "depth2": depth2
                                 })
    return list_of_variants

In [16]:
# define function 'mutate'
def mutate(clone_info, dt, u, last_mutation_ID, DFE): #u=mutation rate
    mutations=clone_info[0] #(mutation_ID, fitness_effect), where mutation_ID uniquely labels each independent mutation to have entered the population
    clone_size=clone_info[1] #integer number of cells comprising the clone

    # prevent a second mutation in an already mutated clone
    if len(mutations) > 1:
        return([], last_mutation_ID)

    number_of_mutations = np.random.poisson(clone_size*u*dt)
    list_of_new_clones=[]
    for i in range(number_of_mutations):
        last_mutation_ID = last_mutation_ID+1
        new_fitness_effect = mutation_fitness(DFE)
        new_clone = [mutations+[(last_mutation_ID, new_fitness_effect)],1]
        list_of_new_clones.append(new_clone)
    return (list_of_new_clones, last_mutation_ID)

## Overall simulation function

- produces file: 'realistic' biobank sequenced at the correct ages

In [20]:
def simulation_choosing_s(s_mCA): #defining a single s (with mu at 4.35e-9) for the simulated mCAs
    
    # PARAMETERS
    # mutation rate and fitness
    u_mCA = 4.35e-9 #  mutation rate per year

    lam = 5 #number of divisions per year
    DFE = [(s_mCA, u_mCA)]   ##, (fitness_effect, mutation_rate))
    u = u_mCA/lam ##+ mutation_rate/lam

    # other parameters
    eq_population_size = 10 ** 5  # number of stem cells (HSCs)
    lifespan = 401  # measured in cell divisions (e.g. 100 = 100 cell divisions) - try not to extend too much otherwise sim stops often (overloaded with variants)
    dt = 1  # measured in units of the overall cell division rate

    last_mutation_ID = 0 #mutation_ID_counter (uniquely labels each indepdent mutation to have entered the population)

    # sequences twice at each call of sequence(), once with depth1 and once with depth2
    depth1 = 500
    depth2 = 150
    # standard deviation in depth
    depthsd1 = 0
    depthsd2 = 0
    
    tau_params = [1, 1, 20, 30] #tau1 and tau2 are both equal to 1, so there is no variation in tau with age
    
    # FOLDERS TO SAVE RESULTS
    folder_path = 'Simulation_results_v2/Simulated_mCAs'
    dirname = f'biobank_sim_test_u={u_mCA}_s_{s_mCA}_N{eq_population_size}_u_mCA={u_mCA}_seed{seed}'
    biobank_variants_name = f'{folder_path}/{dirname}_biobank_variants.csv'
    
    #SIMULATION

    population_dict = {}
    clone_histories = {}

    with open(biobank_variants_name, 'w') as biobank_file:
        writer_biobank = csv.writer(biobank_file, delimiter=',')
        writer_biobank.writerow(["person_ID",
                                 "age",
                                 'mutation_ID',
                                 "mutation_type",
                                 "fitness",
                                 "multi_mutant",
                                 "true_freq",
                                 "freq1",
                                 "var_reads1",
                                 "depth1",
                                 "freq2",
                                 "var_reads2",
                                 "depth2"])
        number_alive_at_end = []
        for person_index in range(len(biobank_ages)-1):
            if person_index % 1000 == 0:
                print(person_index)

            person_ID = biobank_ages.pseudo_ID[person_index]
            ukb_blood_draw = biobank_ages['Age'][person_index]

            n0 = eq_population_size  # starting number of cells in first clone
            founding_pop = [ [[(0, 0.0), ], n0] ] 
            #population = list of clone_infos, where clone_info = [[clone_ID], clone size].
            #clone_ID is a list of (mutation_ID, fitness_effect) pairs for all unique mutations that have accumulated in that clone
            development_phase = 1
            t = 0.0

            population = founding_pop
            population_size = n0

            while t < lifespan:
                if population_size < eq_population_size and development_phase == 1:

                    t = t + dt
                    B0 = 1
                    D0 = 0

                    population_size = pop_size(population)

                    new_population = []
                    for clone_info in population:
                        # first put the ID and clone size in our 'clone histories' dictionary
                        clone_ID = tuple(clone_info[0]) #clone_ID is a list of (mutation_ID, fitness_effect) pairs for all unique mutations that have accumulated in that clone
                        clone_size = clone_info[1]
                        if clone_ID not in clone_histories:
                            clone_histories[clone_ID] = {'sizes': [clone_size], 'times': [t]}
                        else:
                            clone_histories[clone_ID]['sizes'].append(clone_size)
                            clone_histories[clone_ID]['times'].append(t)
                            
                        # then update clone sizes/ population list
                        updated_clone = divide(clone_info, dt, B0, D0)
                        (new_clones, new_latest_ID) = mutate(clone_info, dt, u,
                                                             last_mutation_ID, DFE)
                        last_mutation_ID = new_latest_ID
                        new_population = new_population + new_clones
                        if updated_clone[1] > 0:
                            new_population.append(updated_clone)
                    population = new_population

                else:
                    development_phase = 0  # stops the dev phase first time popsize > eq_pop_size

                    t = t + dt
                    tau = tau_of_t(t / 5, *tau_params)
                    B0 = 0.2 / tau
                    D0 = 0.2 / tau

                    population_size = pop_size(population)

                    new_population = []
                    for clone_info in population:

                        clone_ID = tuple(clone_info[0])
                        clone_size = clone_info[1]

                        if person_ID < 500:
                            # first put the ID and clone size in our 'clone histories' dictionary
                            if t % 1 < dt:
                                if clone_ID not in clone_histories:
                                    clone_histories[clone_ID] = {'sizes': [clone_size], 'times': [t],
                                                                 'cell_fraction': [clone_size / population_size]}
                                else:
                                    clone_histories[clone_ID]['sizes'].append(clone_size)
                                    clone_histories[clone_ID]['times'].append(t)
                                    clone_histories[clone_ID]['cell_fraction'].append(clone_size / population_size)

                        # then update clone sizes/ population list
                        updated_clone = divide(clone_info, dt, B0, D0)
                        (new_clones, new_latest_ID) = mutate(clone_info, dt, u, last_mutation_ID, DFE)
                        last_mutation_ID = new_latest_ID
                        new_population = new_population + new_clones
                        if updated_clone[1] > 0:
                            new_population.append(updated_clone)
                    population = new_population

                    ukb_blood_draw_time = int(ukb_blood_draw * lam / dt)

                    # sequence for biobank file
                    if int(t / dt) == ukb_blood_draw_time:
                        biobank_variants = sequence(population, depth1, depthsd1, depth2, depthsd2, s_mCA)
                        for var in biobank_variants:
                            writer_biobank.writerow([person_ID,
                                                     int(t/lam),
                                                     var['mutation_ID'],
                                                     var['mutation_type'],
                                                     np.round(var['fitness'], 3),
                                                     var['multi_mutant'],
                                                     var['true_freq'],
                                                     var['freq1'],
                                                     var['reads1'],
                                                     var['depth1'],
                                                     var['freq2'],
                                                     var['reads2'],
                                                     var['depth2']])

            number_alive_at_end.append(len(population))

    return print('Simulation completed')

In [21]:
def simulation_choosing_s_and_mu(s_mCA, u_mCA): #defining a single s and single mu for the high-mutation rate mCAs
    
    # PARAMETERS
    # mutation rate and fitness
    lam = 5 #number of divisions per year
    DFE = [(s_mCA, u_mCA)]   ##, (fitness_effect, mutation_rate))
    u = u_mCA/lam ##+ mutation_rate/lam

    # other parameters
    eq_population_size = 10 ** 5  # number of stem cells (HSCs)
    lifespan = 401  # measured in cell divisions (e.g. 100 = 100 cell divisions) - try not to extend too much otherwise sim stops often (overloaded with variants)
    dt = 1  # measured in units of the overall cell division rate

    last_mutation_ID = 0 #mutation_ID_counter (uniquely labels each indepdent mutation to have entered the population)

    # sequences twice at each call of sequence(), once with depth1 and once with depth2
    depth1 = 500
    depth2 = 150
    # standard deviation in depth
    depthsd1 = 0
    depthsd2 = 0
    
    tau_params = [1, 1, 20, 30] #tau1 and tau2 are both equal to 1, so there is no variation in tau with age
    
    # FOLDERS TO SAVE RESULTS
    folder_path = 'Simulation_results/Simulated_high_mutation_rate_mCAs'
    dirname = f'biobank_sim_test_u={u_mCA}_s_{s_mCA}_N{eq_population_size}_u_mCA={u_mCA}_seed{seed}'
    biobank_variants_name = f'{folder_path}/{dirname}_biobank_variants.csv'
    
    #SIMULATION

    population_dict = {}
    clone_histories = {}

    with open(biobank_variants_name, 'w') as biobank_file:
        writer_biobank = csv.writer(biobank_file, delimiter=',')
        writer_biobank.writerow(["person_ID",
                                 "age",
                                 'mutation_ID',
                                 "mutation_type",
                                 "fitness",
                                 "multi_mutant",
                                 "true_freq",
                                 "freq1",
                                 "var_reads1",
                                 "depth1",
                                 "freq2",
                                 "var_reads2",
                                 "depth2"])
        number_alive_at_end = []
        for person_index in range(len(biobank_ages)-1):
            if person_index % 1000 == 0:
                print(person_index)

            person_ID = biobank_ages.pseudo_ID[person_index]
            ukb_blood_draw = biobank_ages['Age'][person_index]

            n0 = eq_population_size  # starting number of cells in first clone
            founding_pop = [ [[(0, 0.0), ], n0] ] 
            #population = list of clone_infos, where clone_info = [[clone_ID], clone size].
            #clone_ID is a list of (mutation_ID, fitness_effect) pairs for all unique mutations that have accumulated in that clone
            development_phase = 1
            t = 0.0

            population = founding_pop
            population_size = n0

            while t < lifespan:
                if population_size < eq_population_size and development_phase == 1:

                    t = t + dt
                    B0 = 1
                    D0 = 0

                    population_size = pop_size(population)

                    new_population = []
                    for clone_info in population:
                        # first put the ID and clone size in our 'clone histories' dictionary
                        clone_ID = tuple(clone_info[0]) #clone_ID is a list of (mutation_ID, fitness_effect) pairs for all unique mutations that have accumulated in that clone
                        clone_size = clone_info[1]
                        if clone_ID not in clone_histories:
                            clone_histories[clone_ID] = {'sizes': [clone_size], 'times': [t]}
                        else:
                            clone_histories[clone_ID]['sizes'].append(clone_size)
                            clone_histories[clone_ID]['times'].append(t)
                            
                        # then update clone sizes/ population list
                        updated_clone = divide(clone_info, dt, B0, D0)
                        (new_clones, new_latest_ID) = mutate(clone_info, dt, u,
                                                             last_mutation_ID, DFE)
                        last_mutation_ID = new_latest_ID
                        new_population = new_population + new_clones
                        if updated_clone[1] > 0:
                            new_population.append(updated_clone)
                    population = new_population

                else:
                    development_phase = 0  # stops the dev phase first time popsize > eq_pop_size

                    t = t + dt
                    tau = tau_of_t(t / 5, *tau_params)
                    B0 = 0.2 / tau
                    D0 = 0.2 / tau

                    population_size = pop_size(population)

                    new_population = []
                    for clone_info in population:

                        clone_ID = tuple(clone_info[0])
                        clone_size = clone_info[1]

                        if person_ID < 500:
                            # first put the ID and clone size in our 'clone histories' dictionary
                            if t % 1 < dt:
                                if clone_ID not in clone_histories:
                                    clone_histories[clone_ID] = {'sizes': [clone_size], 'times': [t],
                                                                 'cell_fraction': [clone_size / population_size]}
                                else:
                                    clone_histories[clone_ID]['sizes'].append(clone_size)
                                    clone_histories[clone_ID]['times'].append(t)
                                    clone_histories[clone_ID]['cell_fraction'].append(clone_size / population_size)

                        # then update clone sizes/ population list
                        updated_clone = divide(clone_info, dt, B0, D0)
                        (new_clones, new_latest_ID) = mutate(clone_info, dt, u, last_mutation_ID, DFE)
                        last_mutation_ID = new_latest_ID
                        new_population = new_population + new_clones
                        if updated_clone[1] > 0:
                            new_population.append(updated_clone)
                    population = new_population

                    ukb_blood_draw_time = int(ukb_blood_draw * lam / dt)

                    # sequence for biobank file
                    if int(t / dt) == ukb_blood_draw_time:
                        biobank_variants = sequence(population, depth1, depthsd1, depth2, depthsd2, s_mCA)
                        for var in biobank_variants:
                            writer_biobank.writerow([person_ID,
                                                     int(t/lam),
                                                     var['mutation_ID'],
                                                     var['mutation_type'],
                                                     np.round(var['fitness'], 3),
                                                     var['multi_mutant'],
                                                     var['true_freq'],
                                                     var['freq1'],
                                                     var['reads1'],
                                                     var['depth1'],
                                                     var['freq2'],
                                                     var['reads2'],
                                                     var['depth2']])

            number_alive_at_end.append(len(population))

    return print('Simulation completed')

In [22]:
def simulation_choosing_s_two_s_two_mu(s1_mCA, s2_mCA, u1_mCA, u2_mCA): #for mLOX simulation with 2s and 2mu
    
    # PARAMETERS
    # mutation rate and fitness
    lam = 5 #number of divisions per year
    DFE = [(s1_mCA, u1_mCA), (s2_mCA, u2_mCA)]   ##, (fitness_effect, mutation_rate))
    u = (u1_mCA+u2_mCA)/lam ## total_mutation_rate/lam

    # other parameters
    eq_population_size = 10 ** 5  # number of stem cells (HSCs)
    lifespan = 401  # measured in cell divisions (e.g. 100 = 100 cell divisions) - try not to extend too much otherwise sim stops often (overloaded with variants)
    dt = 1  # measured in units of the overall cell division rate

    last_mutation_ID = 0 #mutation_ID_counter (uniquely labels each indepdent mutation to have entered the population)

    # sequences twice at each call of sequence(), once with depth1 and once with depth2
    depth1 = 500
    depth2 = 150
    # standard deviation in depth
    depthsd1 = 0
    depthsd2 = 0
    
    tau_params = [1, 1, 20, 30] #tau1 and tau2 are both equal to 1, so there is no variation in tau with age
    
    # FOLDERS TO SAVE RESULTS
    folder_path = 'Simulation_results/Simulated_mLOX_with_2s_2mu'
    dirname = f'biobank_sim_test_s1_{s1_mCA}_s2_{s2_mCA}_u1_{u1_mCA}_u2_{u2_mCA}_N{eq_population_size}_seed{seed}'
    biobank_variants_name = f'{folder_path}/{dirname}_biobank_variants.csv'
    
    #SIMULATION

    population_dict = {}
    clone_histories = {}

    with open(biobank_variants_name, 'w') as biobank_file:
        writer_biobank = csv.writer(biobank_file, delimiter=',')
        writer_biobank.writerow(["person_ID",
                                 "age",
                                 'mutation_ID',
                                 "mutation_type",
                                 "fitness",
                                 "multi_mutant",
                                 "true_freq",
                                 "freq1",
                                 "var_reads1",
                                 "depth1",
                                 "freq2",
                                 "var_reads2",
                                 "depth2"])
        number_alive_at_end = []
        for person_index in range(len(biobank_ages)-1):
            if person_index % 1000 == 0:
                print(person_index)
            
            person_ID = biobank_ages.pseudo_ID[person_index]
            ukb_blood_draw = biobank_ages['Age'][person_index]

            n0 = eq_population_size  # starting number of cells in first clone
            founding_pop = [ [[(0, 0.0), ], n0] ] 
            #population = list of clone_infos, where clone_info = [[clone_ID], clone size].
            #clone_ID is a list of (mutation_ID, fitness_effect) pairs for all unique mutations that have accumulated in that clone
            development_phase = 1
            t = 0.0

            population = founding_pop
            population_size = n0

            while t < lifespan:
                if population_size < eq_population_size and development_phase == 1:

                    t = t + dt
                    B0 = 1
                    D0 = 0

                    population_size = pop_size(population)

                    new_population = []
                    for clone_info in population:
                        # first put the ID and clone size in our 'clone histories' dictionary
                        clone_ID = tuple(clone_info[0]) #clone_ID is a list of (mutation_ID, fitness_effect) pairs for all unique mutations that have accumulated in that clone
                        clone_size = clone_info[1]
                        if clone_ID not in clone_histories:
                            clone_histories[clone_ID] = {'sizes': [clone_size], 'times': [t]}
                        else:
                            clone_histories[clone_ID]['sizes'].append(clone_size)
                            clone_histories[clone_ID]['times'].append(t)
                            
                        # then update clone sizes/ population list
                        updated_clone = divide(clone_info, dt, B0, D0)
                        (new_clones, new_latest_ID) = mutate(clone_info, dt, u, last_mutation_ID, DFE)
                        last_mutation_ID = new_latest_ID
                        new_population = new_population + new_clones
                        if updated_clone[1] > 0:
                            new_population.append(updated_clone)
                    population = new_population

                else:
                    development_phase = 0  # stops the dev phase first time popsize > eq_pop_size

                    t = t + dt
                    tau = tau_of_t(t / 5, *tau_params)
                    B0 = 0.2 / tau
                    D0 = 0.2 / tau

                    population_size = pop_size(population)

                    new_population = []
                    for clone_info in population:

                        clone_ID = tuple(clone_info[0])
                        clone_size = clone_info[1]

                        if person_ID < 500:
                            # first put the ID and clone size in our 'clone histories' dictionary
                            if t % 1 < dt:
                                if clone_ID not in clone_histories:
                                    clone_histories[clone_ID] = {'sizes': [clone_size], 'times': [t],
                                                                 'cell_fraction': [clone_size / population_size]}
                                else:
                                    clone_histories[clone_ID]['sizes'].append(clone_size)
                                    clone_histories[clone_ID]['times'].append(t)
                                    clone_histories[clone_ID]['cell_fraction'].append(clone_size / population_size)

                        # then update clone sizes/ population list
                        updated_clone = divide(clone_info, dt, B0, D0)
                        (new_clones, new_latest_ID) = mutate(clone_info, dt, u, last_mutation_ID, DFE)
                        last_mutation_ID = new_latest_ID
                        new_population = new_population + new_clones
                        if updated_clone[1] > 0:
                            new_population.append(updated_clone)
                    population = new_population

                    ukb_blood_draw_time = int(ukb_blood_draw * lam / dt)

                    # sequence for biobank file
                    if int(t / dt) == ukb_blood_draw_time:
                        biobank_variants = sequence_s1s2(population, depth1, depthsd1, depth2, depthsd2, s1_mCA, s2_mCA)
                        for var in biobank_variants:
                            writer_biobank.writerow([person_ID,
                                                     int(t/lam),
                                                     var['mutation_ID'],
                                                     var['mutation_type'],
                                                     np.round(var['fitness'], 3),
                                                     var['multi_mutant'],
                                                     var['true_freq'],
                                                     var['freq1'],
                                                     var['reads1'],
                                                     var['depth1'],
                                                     var['freq2'],
                                                     var['reads2'],
                                                     var['depth2']])

            number_alive_at_end.append(len(population))

    return print('Simulation completed')

# Generate the simulated data

## Simulated mCAs (each simulation with a different fitness effect)

In [1]:
for fitness in range(5, 36):
    if fitness <10:
        simulation_s = float('0.0'+str(fitness*100+1)) #e.g. 5 -> 0.0501
    else:
        simulation_s = float('0.'+str(fitness*100+1)) #e.g. 11 -> 0.1101
    print('fitness = '+str(simulation_s))
    simulation_choosing_s(simulation_s)
    print()

## Simulated high mutation rate mCAs (each simulation with a different fitness and mu)

In [47]:
simulation_choosing_s_and_mu(0.101, 1e-6) #mutation rate 1e-6, fitness = ~10%

In [48]:
simulation_choosing_s_and_mu(0.101, 1e-5) #mutation rate 1e-5, fitness = ~10%

In [49]:
simulation_choosing_s_and_mu(0.101, 1e-4) #mutation rate 1e-4, fitness = ~10%

## Simulated mLOX with 2 different fitness effects and 2 different mutation rates

In [50]:
simulation_choosing_s_two_s_two_mu(0.08, 0.22, 3e-6, 3e-8)